In [2]:
import os
import geopandas as gpd
import requests
import json
from shapely.geometry import shape, LineString

# 🗂 Set output folder
output_folder = r"C:\Users\91956\Downloads\TN_Wetlands_OSM"
os.makedirs(output_folder, exist_ok=True)

#  Load India States Shapefile
india_states = gpd.read_file(r"C:/Users/91956/Downloads/India_shp/in/in.shp")  # Change to your shapefile path

# Overpass API Query Template
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
way["natural"="wetland"]({bbox});
(._;>;);
out body;
"""

def get_osm_data(bbox):
    """Fetch roads data from Overpass API within the given bounding box."""
    bbox_str = f"{bbox[1]},{bbox[0]},{bbox[3]},{bbox[2]}"  # Format: (S, W, N, E)
    query = overpass_query.replace("{bbox}", bbox_str)
    response = requests.get(overpass_url, params={"data": query})
    if response.status_code == 200:
        return response.json()
    else:
        print(f"⚠️ Failed to fetch data: {response.status_code}")
        return None

def parse_osm_data(osm_data):
    """Convert OSM JSON data into a GeoDataFrame."""
    elements = osm_data.get("elements", [])
    lines = []
    
    # Store nodes with their coordinates
    nodes = {el["id"]: (el["lon"], el["lat"]) for el in elements if el["type"] == "node"}
    
    # Extract ways (lines)
    for el in elements:
        if el["type"] == "way" and "nodes" in el:
            coords = [nodes[node_id] for node_id in el["nodes"] if node_id in nodes]
            if len(coords) > 1:
                lines.append(LineString(coords))
    
    # Create GeoDataFrame
    gdf = gpd.GeoDataFrame(geometry=lines, crs="EPSG:4326")
    return gdf

# 🔄 Loop through each state and process data
for _, state in india_states.iterrows():
    state_name = state["name"]  # Adjust column name if different
    bbox = state.geometry.bounds  # Get bounding box (minx, miny, maxx, maxy)

    print(f"📡 Fetching OSM data for {state_name}...")
    osm_data = get_osm_data(bbox)
    
    if osm_data:
        power_lines = parse_osm_data(osm_data)
        
        # Clip to state boundary
        power_lines = power_lines[power_lines.intersects(state.geometry)]
        
        # Save as Shapefile
        if not power_lines.empty:
            output_path = os.path.join(output_folder, f"NH_{state_name.replace(' ', '_')}.shp")
            power_lines.to_file(output_path)
            print(f"✅ Saved: {output_path}")
        else:
            print(f"⚠️ No power lines found for {state_name}")

print("🎯 Extraction complete!")


📡 Fetching OSM data for Andaman and Nicobar...
✅ Saved: C:\Users\91956\Downloads\TN_Wetlands_OSM\NH_Andaman_and_Nicobar.shp
📡 Fetching OSM data for Telangana...
✅ Saved: C:\Users\91956\Downloads\TN_Wetlands_OSM\NH_Telangana.shp
📡 Fetching OSM data for Andhra Pradesh...
✅ Saved: C:\Users\91956\Downloads\TN_Wetlands_OSM\NH_Andhra_Pradesh.shp
📡 Fetching OSM data for Arunachal Pradesh...
✅ Saved: C:\Users\91956\Downloads\TN_Wetlands_OSM\NH_Arunachal_Pradesh.shp
📡 Fetching OSM data for Assam...
✅ Saved: C:\Users\91956\Downloads\TN_Wetlands_OSM\NH_Assam.shp
📡 Fetching OSM data for Bihar...
✅ Saved: C:\Users\91956\Downloads\TN_Wetlands_OSM\NH_Bihar.shp
📡 Fetching OSM data for Chandigarh...
✅ Saved: C:\Users\91956\Downloads\TN_Wetlands_OSM\NH_Chandigarh.shp
📡 Fetching OSM data for Chhattisgarh...
✅ Saved: C:\Users\91956\Downloads\TN_Wetlands_OSM\NH_Chhattisgarh.shp
📡 Fetching OSM data for DÄdra and Nagar Haveli and DamÄn and Diu...
✅ Saved: C:\Users\91956\Downloads\TN_Wetlands_OSM\NH_DÄdra_

In [2]:
import os
import geopandas as gpd

# Folder where all state shapefiles are saved
input_folder = r"C:\Users\91956\Downloads\roads_teritary"


# Output merged shapefile
output_shapefile = r"C:\Users\91956\Downloads\roads_teritary\merged"

# List to store individual GeoDataFrames
gdfs = []

# Loop through all shapefiles in the folder
for file in os.listdir(input_folder):
    if file.endswith(".shp"):
        filepath = os.path.join(input_folder, file)
        gdf = gpd.read_file(filepath)
        gdfs.append(gdf)
        print(f"📂 Loaded: {file}")

# Merge all GeoDataFrames into one
if gdfs:
    merged_gdf = gpd.pd.concat(gdfs, ignore_index=True)
    
    # Save the merged shapefile
    merged_gdf.to_file(output_shapefile)
    print(f"✅ Merged shapefile saved: {output_shapefile}")
else:
    print("⚠️ No shapefiles found in the folder!")


📂 Loaded: NH_Andaman_and_Nicobar.shp
📂 Loaded: NH_Andhra_Pradesh.shp
📂 Loaded: NH_Arunachal_Pradesh.shp
📂 Loaded: NH_Assam.shp
📂 Loaded: NH_Bihar.shp
📂 Loaded: NH_Chandigarh.shp
📂 Loaded: NH_Chhattisgarh.shp
📂 Loaded: NH_Delhi.shp
📂 Loaded: NH_DÄdra_and_Nagar_Haveli_and_DamÄn_and_Diu.shp
📂 Loaded: NH_Goa.shp
📂 Loaded: NH_Gujarat.shp
📂 Loaded: NH_Haryana.shp
📂 Loaded: NH_Himachal_Pradesh.shp
📂 Loaded: NH_Jammu_and_Kashmir.shp
📂 Loaded: NH_Jharkhand.shp
📂 Loaded: NH_Karnataka.shp
📂 Loaded: NH_Kerala.shp
📂 Loaded: NH_Ladakh.shp
📂 Loaded: NH_Lakshadweep.shp
📂 Loaded: NH_Madhya_Pradesh.shp
📂 Loaded: NH_Maharashtra.shp
📂 Loaded: NH_Manipur.shp
📂 Loaded: NH_Meghalaya.shp
📂 Loaded: NH_Mizoram.shp
📂 Loaded: NH_Nagaland.shp
📂 Loaded: NH_Orissa.shp
📂 Loaded: NH_Puducherry.shp
📂 Loaded: NH_Punjab.shp
📂 Loaded: NH_Rajasthan.shp
📂 Loaded: NH_Sikkim.shp
📂 Loaded: NH_Tamil_Nadu.shp
📂 Loaded: NH_Telangana.shp
📂 Loaded: NH_Tripura.shp
📂 Loaded: NH_Uttaranchal.shp
📂 Loaded: NH_Uttar_Pradesh.shp
📂 Loade